In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cartiflette import carti_download
import plotly.express as px

## Import Données

In [ ]:
df_patientele = pd.read_csv("Bases de données/patientele.csv", sep=";")
df_patientele['nombre_patients_uniques'] = pd.to_numeric(df_patientele['nombre_patients_uniques'],errors='coerce')
df_patientele.columns = df_patientele.columns.str.replace("\ufeff", "", regex=False).str.strip()

df_patientele_2017 = df_patientele[(df_patientele['annee'] == 2017) & (df_patientele['profession_sante'] == "Médecins généralistes (hors médecins à expertise particulière - MEP)")]
df_patientele_2017 = df_patientele_2017[["departement","libelle_departement","patients_uniques_integer"]]
df_patientele_2017.columns = ["Code Département","Département","Nb de patients unique par spécialiste"]
df_patientele_2017.head() 

In [ ]:
df_Indicateurs_département = pd.read_excel("Bases de données/data (1).xlsx")
#df_Indicateurs_département = df_Indicateurs_département[["dep", "codgeo", "libgeo", "p21_pop"]]
#df_commune.reset_index(drop=True, inplace=True)
df_Indicateurs_département.columns = ["Code Département", "Libellé Département", "Nombre de décès domiciliés 2024", "Salaire net EQTP mensuel moyen 2023","Espérance de vie des hommes à 60 ans 2024",	"Espérance de vie des femmes à 60 ans","Taux d'activité","Taux de chômage annuel moyen 2024","Part des résidences principales en suroccupation","Taux de pauvreté - familles monoparentales 2021","Taux de pauvreté 2021","Médiane du niveau de vie 2021","Taux brut de mortalité 2024","Taux de natalité 2024"]
df_Indicateurs_département = df_Indicateurs_département.sort_values(by="Code commune INSEE")

df_Indicateurs_département.head()